In [60]:
%load_ext autoreload
%autoreload 2
import polars as pl
from polars import col
import datetime as dt
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
df_prices = pl.read_delta(
    "s3://dev-energy-market-bronze/aemo/vicgas/bronze_int310_v4_price_and_withdrawals_1"
)

In [62]:
df_prices.head()

gas_date,schedule_interval,transmission_id,sched_inj_gj,sched_wdl_gj,price_value,administered_price,actual_wdl_gj,actual_inj_gj,source_file
str,i64,i64,f64,f64,f64,str,f64,f64,str
"""05 Jun 2025""",3,270309,904814.931619,913010.0,11.2488,null,818221.489814,822912.154046,"""s3://dev-energy-market-landing…"
"""05 Jun 2025""",5,270337,926880.153934,984447.842132,9.5012,null,818221.489814,822912.154046,"""s3://dev-energy-market-landing…"
"""04 Jun 2025""",1,270263,916250.62963,934415.62964,11.3348,null,864465.746658,839815.859459,"""s3://dev-energy-market-landing…"
"""04 Jun 2025""",2,270268,911425.469477,928876.0,11.5044,null,864465.746658,839815.859459,"""s3://dev-energy-market-landing…"
"""04 Jun 2025""",4,270283,904260.903703,931893.031704,11.4025,null,864465.746658,839815.859459,"""s3://dev-energy-market-landing…"


In [63]:
df_cleansed = (
    df_prices.with_columns(col.gas_date.str.to_date("%d %b %Y"))
    .with_columns(
        pl.when(col.schedule_interval == 1)
        .then(pl.lit(dt.timedelta(hours=6)))
        .when(col.schedule_interval == 2)
        .then(pl.lit(dt.timedelta(hours=10)))
        .when(col.schedule_interval == 3)
        .then(pl.lit(dt.timedelta(hours=14)))
        .when(col.schedule_interval == 4)
        .then(pl.lit(dt.timedelta(hours=18)))
        .when(col.schedule_interval == 5)
        .then(pl.lit(dt.timedelta(hours=22)))
        .alias("hour")
    )
    .with_columns(scheduling_horizon=(col.gas_date.cast(pl.Datetime) + col.hour))
    .sort("scheduling_horizon")
)
df_cleansed.head()

gas_date,schedule_interval,transmission_id,sched_inj_gj,sched_wdl_gj,price_value,administered_price,actual_wdl_gj,actual_inj_gj,source_file,hour,scheduling_horizon
date,i64,i64,f64,f64,f64,str,f64,f64,str,duration[μs],datetime[μs]
2024-05-01,1,260097,642555.698018,654292.866657,12.1688,null,594454.51222,586458.37271,"""s3://dev-energy-market-landing…",6h,2024-05-01 06:00:00
2024-05-01,2,260105,647556.0,663555.42191,11.8655,null,594454.51222,586458.37271,"""s3://dev-energy-market-landing…",10h,2024-05-01 10:00:00
2024-05-01,3,260109,641680.11895,673127.99997,11.7489,null,594454.51222,586458.37271,"""s3://dev-energy-market-landing…",14h,2024-05-01 14:00:00
2024-05-01,4,260114,634867.280143,681308.666627,11.6538,null,594454.51222,586458.37271,"""s3://dev-energy-market-landing…",18h,2024-05-01 18:00:00
2024-05-01,5,260121,625483.238399,680242.797583,11.0422,null,594454.51222,586458.37271,"""s3://dev-energy-market-landing…",22h,2024-05-01 22:00:00


In [64]:
latest_date = df_cleansed.select(col.scheduling_horizon).max().cast(pl.Date).item()
earliest_date  =latest_date - dt.timedelta(days=30)

print(earliest_date, latest_date)

2025-06-03 2025-07-03


In [87]:
fig = px.line(
    df_cleansed,
    title="Gas Prices Per Scheduling Horizon",
    x="scheduling_horizon",
    y="price_value",
    width=1000,
    height=800,
)


# Add dollar units to the y-axis
fig.update_layout(
    yaxis=dict(tickprefix="$", title="Price (AUD)"),
    xaxis=dict(
        title="Scheduling Horizon",
        tickformat="%Y-%m-%d",
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
        range=[str(earliest_date), str(latest_date)],
        showspikes=True,
    ),
    hovermode="x",
)

# Customize hover to show full datetime
fig.update_traces(
    mode="lines",
    hovertemplate="Scheduling Horizon: %{x|%Y-%m-%d %H:%M}<br>Price: $%{y:.2f}<extra></extra>",
)